# <B> Setup for SageMaker BYOM examples </B>
* Container: conda_python3

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. Set roles

In [2]:
from sagemaker import get_execution_role

In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20230714T100409


### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [4]:
from utils.iam import iam_handler

In [5]:
iam = iam_handler()

In [6]:
strID = "DJ"
strPrefix = f'{strID}-SM-IMD'

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>
> EventBridge <BR>
> Lambda

In [7]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    #"arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    #"arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    #"arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 1.3 Create role for lambda

In [8]:
listService = ["lambda.amazonaws.com"]
strRoleName = "-".join([strPrefix, "LabmdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
]
strDescription="Role for Lambda to call SageMaker functions'"

In [9]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

== CREATE ROLE ==
  Service name: ['lambda.amazonaws.com'], 
  Role name: DJ-SM-IMD-LabmdaRole, 
  Policys: ['arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole', 'arn:aws:iam::aws:policy/AWSLambda_FullAccess', 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'], 
  Role ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-LabmdaRole
== COMPLETED ==


### 1.4 role-codebuild

In [10]:
listService = ["codebuild.amazonaws.com", "sagemaker.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodeBuildRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [11]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Service name: ['codebuild.amazonaws.com', 'sagemaker.amazonaws.com'], 
  Role name: DJ-SM-IMD-CodeBuildRole, 
  Policys: ['arn:aws:iam::aws:policy/AWSCodeCommitFullAccess', 'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess', 'arn:aws:iam::aws:policy/AmazonS3FullAccess', 'arn:aws:iam::aws:policy/CloudWatchFullAccess', 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess', 'arn:aws:iam::aws:policy/AWSLambda_FullAccess', 'arn:aws:iam::aws:policy/AmazonSSMFullAccess'], 
  Role ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-CodeBuildRole
== COMPLETED ==
CODE-BUILD-ROLE-ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-CodeBuildRole


### 1.5 role-codepipeline

In [12]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodePipelineRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [13]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Service name: ['codepipeline.amazonaws.com', 'codebuild.amazonaws.com'], 
  Role name: DJ-SM-IMD-CodePipelineRole, 
  Policys: ['arn:aws:iam::aws:policy/AWSCodeCommitFullAccess', 'arn:aws:iam::aws:policy/AmazonS3FullAccess', 'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess', 'arn:aws:iam::aws:policy/AWSCodeDeployFullAccess', 'arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess'], 
  Role ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-CodePipelineRole
== COMPLETED ==
CODE-PIPELINE-ROLE-ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-CodePipelineRole


### 1.6 role-eventbridge

In [14]:
listService = ["events.amazonaws.com"]
strRoleName = "-".join([strPrefix, "EventBridgeRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
]
strDescription="Role for EventBridge"

In [15]:
eventBridgeRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {eventBridgeRoleArn}")

== CREATE ROLE ==
  Service name: ['events.amazonaws.com'], 
  Role name: DJ-SM-IMD-EventBridgeRole, 
  Policys: ['arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'], 
  Role ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-EventBridgeRole
== COMPLETED ==
CODE-PIPELINE-ROLE-ARN: arn:aws:iam::147292109112:role/DJ-SM-IMD-EventBridgeRole


## 2. Set default parameters

In [16]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [17]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

In [18]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"]), value=eventBridgeRoleArn, overwrite=True)

'Store suceess'

In [19]:
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))
print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
#print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
#print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))
print ("CODE-EVENTBRIDGE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"])))

REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-147292109112
SAGEMAKER-ROLE-ARN:  arn:aws:iam::147292109112:role/service-role/AmazonSageMaker-ExecutionRole-20230714T100409
ACCOUNT-ID:  147292109112
LAMBDA-ROLE-ARN:  arn:aws:iam::147292109112:role/DJ-SM-IMD-LabmdaRole
CODE-EVENTBRIDGE-ROLE-ARN:  arn:aws:iam::147292109112:role/DJ-SM-IMD-EventBridgeRole
